In [148]:
import requests
from bs4 import BeautifulSoup
from tld import get_tld
import re
from tqdm import tqdm

def findWords(list_of_sentences,list_of_keywords):

    regex = re.compile('.*('+'|'.join(list_of_keywords)+').*',re.IGNORECASE)
    return list(filter(regex.match,list_of_sentences))

def findWordsStrict(list_of_sentences,list_of_keywords):

    regex = re.compile(r'.*\W('+'|'.join(list_of_keywords)+r')\W.*',re.IGNORECASE)
    return list(filter(regex.match,list_of_sentences))

class website():
    def __init__(self,url):

        self.domain = get_tld(url,as_object = True).fld
        self.site = BeautifulSoup(requests.get(url).text,'html.parser')

    def getElement(self,elements=['h1','h2','h3','h4','h5','h6','li','strong','a','span','div','p']):

        self.contentByElement = {}
        self.content = []
        self.allIntHref = []
        self.allExtHref = []
        self.cleanContent = []

        for element in elements:
            self.contentByElement[element] = self.site.find_all(element)
        
        self.content.extend(self.site.find_all(elements))
        for content in self.content:
            try: self.cleanContent.append(content['href'])
            except: pass
            self.cleanContent.append(content.get_text())
        
        for a in self.site.find_all('a'):
            href = a['href']
            if href[0] == '/':
                self.allIntHref.append('http://'+self.domain+href)
            elif href[0] != '#':
                self.allExtHref.append(href)
                
        self.allIntHref = list(set(self.allIntHref))
        self.allExtHref = list(set(self.allExtHref))   

    def relevant(self,keywords = []):

        regex = re.compile('('+'|'.join(keywords)+')',re.IGNORECASE)
        self.textByElement = {}
        self.text = []
        self.intHref = []
        self.extHref = []

        try:
            cbe = self.contentByElement
            for byElement in cbe:
                self.textByElement[byElement] = []
                for content in cbe[byElement]:
                    p = content.get_text()
                    try:
                        if regex.search(p):
                            self.text.append(content.get_text().strip('\n'))
                            self.textByElement[byElement].append(content.get_text().strip('\n'))
                            href = content.a['href']
                            if href[0] == '/':
                                self.intHref.append('http://'+self.domain+href)
                            elif href[0] != '#':
                                self.extHref.append(href)
                    except: pass
        except: pass

        self.intHref = list(set(self.intHref))
        self.extHref = list(set(self.extHref))

starting_domain = 'http://wikipedia.org/wiki/'

def spidermanW(keywords,limit=300,subwords=[]):

    urls = [starting_domain + '_'.join(keyword.split()) for keyword in keywords]
    textByUrl = {}
    text = []

    for url in tqdm(urls):

        try:
            site = website(url)
            site.getElement()
            site.relevant(keywords)
            text.extend(findWordsStrict(site.text,subwords))
            textByUrl[url] = findWordsStrict(site.text,subwords)

            for link in site.intHref:
                if link not in urls and len(urls) < limit:
                    urls.append(link)

            for link in site.extHref:
                if link not in urls and len(urls) < limit:
                    urls.append(link)
        except: pass

    return text,textByUrl

In [155]:
L = ['']
for i in range(2,23):
    L.append('page/%d/' % i)
# site = website('https://jlptsensei.com/complete-jlpt-grammar-list/')
site.getElement()
site.relevant()
site.cleanContent

['      N1 How to Pass JLPT N1 – Study GuideJLPT N1 Grammar ListJLPT N1 Practice Test – Free DownloadN2 How to Pass JLPT N2 – Study GuideJLPT N2 Grammar ListJLPT N2 Vocabulary ListJLPT N2 Practice Test – Free DownloadN3 How to Pass JLPT N3 – Study GuideJLPT N3 Grammar ListJLPT N3 Kanji ListJLPT N3 Vocabulary ListJLPT N3 Practice Test – Free DownloadN4 How to Pass JLPT N4 – Study GuideJLPT N4 Grammar ListJLPT N4 Kanji ListJLPT N4 Vocabulary ListJLPT N4 Practice Test – Free DownloadN5 How to Pass JLPT N5 – Study GuideJLPT N5 Grammar ListJLPT N5 Kanji ListJLPT N5 Vocabulary ListJLPT N5 Practice Test – Free DownloadJLPT Info What is the JLPT?JLPT Info & NewsJLPT Pass Rate StatisticsComplete JLPT Grammar ListLearn JapaneseRecommended ResourcesDownloads ebooksFlashcardsAll Downloads  Search     Grammar (all levels)N1 Grammar ListN2 Grammar ListN3 Grammar ListN4 Grammar ListN5 Grammar ListComplete JLPT Grammar ListJapanese Grammar Database to Pass the JLPTThis is our complete JLPT grammar lis

In [166]:
for a in site.site.contents:
    print(a)

html
[if lt IE 7]><html lang="en-US" class="no-js lt-ie9 lt-ie8 lt-ie7"><![endif]
[if (IE 7)&!(IEMobile)]><html lang="en-US" class="no-js lt-ie9 lt-ie8"><![endif]
[if (IE 8)&!(IEMobile)]><html lang="en-US" class="no-js lt-ie9"><![endif]
[if gt IE 8]><!
<html class="no-js" lang="en-US"><!--<![endif]--><head><meta charset="utf-8"/>
<script data-cfasync="false" data-ezscrex="false" data-pagespeed-no-defer="">var __ez=__ez||{};__ez.stms=Date.now();__ez.evt={};__ez.script={};__ez.ck=__ez.ck||{};__ez.template={};__ez.template.isOrig=false;__ez.queue=(function(){var count=0,incr=0,items=[],timeDelayFired=false,hpItems=[],lpItems=[],allowLoad=true;var obj={func:function(name,funcName,parameters,isBlock,blockedBy,deleteWhenComplete,proceedIfError){var self=this;this.name=name;this.funcName=funcName;this.parameters=parameters===null?null:(parameters instanceof Array)?parameters:[parameters];this.isBlock=isBlock;this.blockedBy=blockedBy;this.deleteWhenComplete=deleteWhenComplete;this.isError=fals

In [118]:
','.join([])

''

In [79]:
# china = website('https://news.baidu.com/')
china.site
site.getElement()
site.relevant([''])
site.extHref

[]

In [49]:
site.getElement()
site.relevant([''])
for i in site.intHref:
    try: print(re.findall('url\?q=(.*?)&sa',i)[0])
    except: pass

https://www.donaldjtrump.com/
https://www.theguardian.com/us-news/donaldtrump
https://twitter.com/realDonaldTrump/status/1294487623285837824%3Fref_src%3Dtwsrc%255Egoogle%257Ctwcamp%255Eserp%257Ctwgr%255Etweet
https://en.wikipedia.org/wiki/Donald_Trump%23Personal_life
https://twitter.com/realDonaldTrump%3Fref_src%3Dtwsrc%255Egoogle%257Ctwcamp%255Eserp%257Ctwgr%255Eauthor
https://edition.cnn.com/specials/politics/president-donald-trump-45
https://en.wikipedia.org/wiki/The_Wharton_School
https://en.wikipedia.org/wiki/Independent_politician
https://en.wikipedia.org/wiki/Republican_Party_(United_States)
https://en.wikipedia.org/wiki/Donald_Trump
https://en.wikipedia.org/wiki/Mike_Pence
https://en.wikipedia.org/wiki/Reform_Party_of_the_United_States_of_America
https://www.independent.co.uk/topic/DonaldTrump
http://t1.gstatic.com/images%3Fq%3Dtbn:ANd9GcQQn6_Hz9zTckXYuOa1biiMhulnHv6pKtadAFcdg79yocrL3Y29
https://en.wikipedia.org/wiki/Economics
https://en.wikipedia.org/wiki/Donald_Trump
https://

In [151]:
site = spidermanW(['countries'],limit=10,subwords=['china'])

100%|██████████| 1/1 [00:00<00:00,  2.23it/s]


In [7]:
def spidermanG(keywords,subwords=[]):
    starting_domain = 'http://wikipedia.org/wiki/'
    urls = [starting_domain + '_'.join(keyword.split()) for keyword in keywords]
    texts = []
    for url in tqdm(urls):
        site = website(url)
        site.relevant(keywords)
        site.clean_href()
        texts.extend(site.rp)
        for link in site.href:
            link = 'http://'+site.domain+link
            if link not in urls and len(urls) < 200:
                urls.append(link)
    return find_words(texts,subwords)